* 그룹 연산: 분리-적용-결합
  1. Series, DataFrame과 같은 pandas객체 혹은 다른 객체에 들어있는 데이터를 <b>하나 이상의 키를 기준으로 분리</b>한다.
  2. 함수를 각 그룹에 적용시켜 새로운 값을 얻어낸다.
  3. 함수 적용한 결과를 하나의 객체로 결합한다.
    * 이 때, 결과를 담는 객체는 보통 데이터에 어떤 연산을 했는지에 따라 결정된다.

In [1]:
import pandas as pd
import numpy as np

# 1. GROUPBY 메카닉

In [2]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                  'key2': ['one', 'two', 'one', 'two', 'one'],
                  'data1': np.random.randn(5),
                  'data2': np.random.randn(5)})

In [3]:
df

,key1,key2,data1,data2
0,a,one,0.616777,0.143922
1,a,two,-0.678890,-0.071281
2,b,one,-0.493890,1.212174
3,b,two,-1.654307,0.522999
4,a,one,-1.132079,0.330469


In [4]:
# 위의 데이터를 key1으로 묶고 각 그룹에서 data1의 평균을 구해보자
# 여러 방법 중 data1에 대해 groupby 메서드 호출 후 key1 컬럼을 넘기는 방식을 사용한다
grouped = df['data1'].groupby(df['key1'])

In [5]:
# grouped 변수는 GroupBy 객체
# df['key1']로 참조되는 중간값에 대한 것 외에는 아무 것도 계산되지 않은 객체
# 그룹 연산을 위해 필요한 모든 정보를 가지고 있어 각 그룹에 어떤 연산을 적용할 수 있게 해 줌!
grouped

In [6]:
# 예) 그룹별 평균을 구하려면 GroupBy 객체의 mean 메서드 사용
# 중요한 점은 데이터가 "그룹 색인에 따라 수집"되고 key1 컬럼에 있는 "유일한 값으로 색인"되는 새로운 Series 객체가 생성된다는 것!
# 새롭게 생성된 Series 객체의 색인은 'key1'인데, 그 이유는 DataFrame 컬럼인 df['key1'] 때문!
grouped.mean()

key1
a   -0.398064
b   -1.074098
Name: data1, dtype: float64

In [7]:
# 여러 개의 배열을 리스트로 넘겼다면 다른 결과를 얻었을 것!
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [8]:
means

key1  key2
a     one    -0.257651
      two    -0.678890
b     one    -0.493890
      two    -1.654307
Name: data1, dtype: float64

In [157]:
type(means)

pandas.core.series.Series

In [9]:
# 여기서는 데이터를 두 개의 색인으로 묶었고, 그 결과 계층적 색인을 가지는 Series를 얻을 수 있음
means.unstack()

key2,one,two
key1,,
a,-0.257651,-0.678890
b,-0.493890,-1.654307


In [10]:
# 이 예제에서는 그룹의 색인 모두 Series 객체인데, 길이만 같다면 어떤 배열이라도 상관없다.
# Series 객체요..? ndarray 아닌가여?
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [11]:
type(states)

numpy.ndarray

In [12]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [13]:
states

array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'], dtype='<U10')

In [14]:
df['data1'].groupby([states, years]).mean()

California  2005   -0.678890
            2006   -0.493890
Ohio        2005   -0.518765
            2006   -1.132079
Name: data1, dtype: float64

In [15]:
# 한 그룹으로 묶을 정보는 주로 같은 DataFrame 안에서 찾는데, 이 경우 컬럼 이름을 넘겨 그룹의 색인으로 사용할 수 있다.
# 해당 예제의 경우 key2 컬럼이 결과에서 빠져있음: df['key2']는 숫자 데이터가 아니기 때문에 "성가신 컬럼"이라 부르며 결과에서 제외시킴
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.398064,0.134370
b,-1.074098,0.867587


In [16]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.257651  0.237196
     two  -0.678890 -0.071281
b    one  -0.493890  1.212174
     two  -1.654307  0.522999

In [17]:
# groupby()를 쓰는 목적과 별개로 유용한 메서드는 그룹의 크기를 담고 있는 Series를 반환하는 size 메서드이다.
# 그룹 색인에서 누락된 값은 결과에서 제외된다는 것을 기억!
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [18]:
df

,key1,key2,data1,data2
0,a,one,0.616777,0.143922
1,a,two,-0.678890,-0.071281
2,b,one,-0.493890,1.212174
3,b,two,-1.654307,0.522999
4,a,one,-1.132079,0.330469


In [19]:
type(df.groupby(['key1', 'key2']).size())

pandas.core.series.Series

## MARK: 그룹간 순회하기
* GroupBy 객체는 이터레이션을 지원
* 그룹 이름과 그에 따른 데이터 묶음을 튜플로 반환


In [20]:
for name, group in df.groupby('key1'):
    print(name)

a
b


In [21]:
for name, group in df.groupby('key1'):
    print(group)

  key1 key2     data1     data2
0    a  one  0.616777  0.143922
1    a  two -0.678890 -0.071281
4    a  one -1.132079  0.330469
  key1 key2     data1     data2
2    b  one -0.493890  1.212174
3    b  two -1.654307  0.522999


In [22]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.616777  0.143922
1    a  two -0.678890 -0.071281
4    a  one -1.132079  0.330469
b
  key1 key2     data1     data2
2    b  one -0.493890  1.212174
3    b  two -1.654307  0.522999


In [23]:
# 색인이 여럿 존재하는 경우 첫 번째 원소가 색인값이 됨
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))

('a', 'one')
('a', 'two')
('b', 'one')
('b', 'two')


In [24]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print(group)

  key1 key2     data1     data2
0    a  one  0.616777  0.143922
4    a  one -1.132079  0.330469
  key1 key2    data1     data2
1    a  two -0.67889 -0.071281
  key1 key2    data1     data2
2    b  one -0.49389  1.212174
  key1 key2     data1     data2
3    b  two -1.654307  0.522999


In [25]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.616777  0.143922
4    a  one -1.132079  0.330469
('a', 'two')
  key1 key2    data1     data2
1    a  two -0.67889 -0.071281
('b', 'one')
  key1 key2    data1     data2
2    b  one -0.49389  1.212174
('b', 'two')
  key1 key2     data1     data2
3    b  two -1.654307  0.522999


In [26]:
# 원하는 데이터만 추출 가능
# 아래 코드 한 줄이면 그룹별 데이터를 사전형으로 쉽게 바꾸어서 유용하게 사용 가능
pieces = dict(list(df.groupby('key1')))

In [27]:
pieces

{'a':   key1 key2     data1     data2
 0    a  one  0.616777  0.143922
 1    a  two -0.678890 -0.071281
 4    a  one -1.132079  0.330469,
 'b':   key1 key2     data1     data2
 2    b  one -0.493890  1.212174
 3    b  two -1.654307  0.522999}

In [28]:
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.493890,1.212174
3,b,two,-1.654307,0.522999


In [29]:
# groupby 메서드는 기본적으로 axis=0에 대해 그룹 생성
# 다른 축으로 그룹을 만드는 것도 가능
# df의 컬럼을 dtype에 따라 그룹으로 묶을 수도 있음!
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [30]:
grouped = df.groupby(df.dtypes, axis=1)   # axis=1 열

In [31]:
grouped

In [32]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.616777  0.143922
1 -0.678890 -0.071281
2 -0.493890  1.212174
3 -1.654307  0.522999
4 -1.132079  0.330469
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


## MARK: 컬럼이나 컬럼의 일부 선택하기
* DataFrame에서 만든 GroupBy 객체를 컬럼 이름이나 컬럼 이름이 담긴 배열로 색인하면 수집을 위해 해당 컬럼을 선택하게 됨
<pre>
    df.groupby('key1')['data1']
    df.groupby('key1')[['data2']]
</pre>
* 위 코드는 아래 코드에 대한 Syntactic sugar(코드를 사람이 이해하기 쉽게 재디자인한 코드)로 같은 결과를 반환
<pre>
    df['data1'].groupby(df['key1'])
    df[['data2']].groupby(df['key1'])
</pre>
* 특히 대용량 데이터를 다룰 경우 소수의 컬럼만 집계하고 싶은 경우가 있음. 해당 예제를 함께 살펴보자!

In [33]:
# 위 데이터에서 data2 컬럼에 대해서만 평균을 구하고 결과를 DataFrame으로 받고 싶다면?
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.237196
     two  -0.071281
b    one   1.212174
     two   0.522999

In [34]:
# 호오 반환값이 DataFrame이 맞네유
type(df.groupby(['key1', 'key2'])[['data2']].mean())

pandas.core.frame.DataFrame

* 색인으로 얻은 객체는 groupby 메서드에 리스트나 배열을 넘겼을 경우 DataFrameGroupBy 객체가 됨 (이름,,,, 정말,,,, 잘 지었네,,,,,^^,,)
* 단일 값으로 하나의 컬럼 이름만 넘겼을 경우 SeriesGroupBy 객체가 됨

In [35]:
# 하나의 컬럼 이름 넘기기('data2')
s_grouped = df.groupby(['key1', 'key2'])['data2']

In [36]:
s_grouped

In [37]:
s_grouped.mean()

key1  key2
a     one     0.237196
      two    -0.071281
b     one     1.212174
      two     0.522999
Name: data2, dtype: float64

## MARK: 사전과 Series에서 그룹핑하기
* 그룹 정보는 배열이 아닌 형태로 존재하기도 함

In [38]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'], 
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [39]:
people.iloc[2:3, [1, 2]] = np.nan     # nan 값 추가

In [40]:
people

,a,b,c,d,e
Joe,0.989621,0.171206,0.120490,1.482108,-0.854458
Steve,-0.886774,1.022987,-0.404978,1.019578,0.263766
Wes,1.175459,NaN,NaN,-0.743461,0.262330
Jim,0.260323,0.896450,-0.723059,-1.255202,0.824109
Travis,-0.983842,0.114243,2.230715,-1.044711,-0.221852


In [41]:
# 이제 각 컬럼을 나타낼 그룹 목록이 있고, 그룹 별로 컬럼의 값을 모두 더한다고 해 보자
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [42]:
# 위의 사전에서 groupby 메서드로 넘길 배열을 뽑아낼 수 있지만 그냥 이 사전을 groupby 메서드로 넘겨보자
# 사용하지 않는 그룹 키도 문제없다는 것을 보이기 위해 'f'도 포함시켰다
by_column = people.groupby(mapping, axis=1)

In [43]:
by_column

In [44]:
by_column.sum()

,blue,red
Joe,1.602598,0.306369
Steve,0.614600,0.399980
Wes,-0.743461,1.437789
Jim,-1.978261,1.980882
Travis,1.186004,-1.091451


In [45]:
# Series에 대해서도 같은 기능 수행 가능. 고정된 크기의 맵이라고 보면 됨.
map_series = pd.Series(mapping)

In [46]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [47]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## MARK: 함수로 그룹핑하기!
* 파이썬 함수를 사용하는 것은 사전이나 Series를 사용해서 그룹을 매핑하는 것 보다 좀 더 일반적인 방법
* 그룹 색인으로 넘긴 함수는 색인값 하나마다 한 번씩 호출되며, 반환값은 그 그룹의 이름으로 사용
* 좀 더 구체적으로 말하자면, 좀 전에 살펴본 예제에서 people DataFrame은 사람의 이름을 색인값으로 사용
* 만약 이름의 길이별로 그룹을 묶고 싶다면 이름의 길이가 담긴 배열을 만드어 넘기는 대신 len 함수를 넘기면 됨

In [48]:
# 그냥 len이라고만 넘기면 되는구나 호오
people.groupby(len).sum()

,a,b,c,d,e
3,2.425404,1.067656,-0.602569,-0.516554,0.231980
5,-0.886774,1.022987,-0.404978,1.019578,0.263766
6,-0.983842,0.114243,2.230715,-1.044711,-0.221852


In [49]:
# 내부적으로는 모두 배열로 변환되므로 함수를 배열, 사전 또는 Series와 섞어 쓰더라도 전혀 문제가 되지 않음
key_list = ['one', 'one', 'one', 'two', 'two']

In [50]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.989621  0.171206  0.120490 -0.743461 -0.854458
  two  0.260323  0.896450 -0.723059 -1.255202  0.824109
5 one -0.886774  1.022987 -0.404978  1.019578  0.263766
6 two -0.983842  0.114243  2.230715 -1.044711 -0.221852

## MARK: 색인 단계로 그룹핑하기
* 계층적으로 색인된 데이터는 축 색인의 단계중 하나를 사용해서 편리하게 집계할 수 있는 기능 제공

In [51]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], [1, 3, 5, 1, 3]],
                                    names=['city', 'tenor'])

In [52]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)

In [53]:
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      0.282993  0.237107 -2.527366 -0.624201 -0.771775
1     -0.641706 -0.259601  1.122869  0.009516  1.032828
2     -1.480172 -0.066534 -0.059179 -0.582491  0.412132
3     -1.328308  0.162783 -1.020888  1.116830 -0.669906

In [54]:
hier_df.groupby(level='city', axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# 2. 데이터 집계
<pre> * <b>데이터 집계</b>
 : 배열로부터 스칼라값을 만들어내는 모든 데이터 변환 작업
</pre>
* 위 예제에서는 mean, count, min, sum을 이용해서 스칼라값을 구함.
* GroupBy 객체에 대해 mean()을 수행하면 어떻게 될까?
* 뿐만 아니라 직접 고안한 집계함수를 사용하고 추가적으로 그룹 객체에 이미 정의된 메서드를 이용해서 사용하는 것도 가능!

In [55]:
df

,key1,key2,data1,data2
0,a,one,0.616777,0.143922
1,a,two,-0.678890,-0.071281
2,b,one,-0.493890,1.212174
3,b,two,-1.654307,0.522999
4,a,one,-1.132079,0.330469


In [56]:
grouped = df.groupby('key1')

In [57]:
# quantile 메서드가 Series나 DataFrame의 컬럼의 변위치를 계산한다는 점을 기억하자
# quantile 메서드는 GroupBy만을 위해 구현되지는 않았지만 Series 메서드이기 때문에 사용 가능하다
# 내부적으로 GroupBy는 Series를 효과적으로 잘게 자르고, 각 조각에 대해 piece.quantile(0.9)를 호출한다.
# 그리고 이 결과들을 모두 하나의 객체로 합쳐서 반환한다.
grouped['data1'].quantile(0.9)

key1
a    0.357644
b   -0.609932
Name: data1, dtype: float64

In [58]:
# 자신만의 데이터 집계함수를 사용하려면 배열의 aggregate나 agg 메서드에 해당 함수를 넘기면 됨
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [59]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.748856,0.401750
b,1.160416,0.689176


In [60]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.398064  0.907619 -1.132079 -0.905484 -0.678890 -0.031056   
b      2.0 -1.074098  0.820538 -1.654307 -1.364202 -1.074098 -0.783994   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.616777   3.0  0.134370  0.201045 -0.071281  0.036321  0.143922   
b    -0.493890   2.0  0.867587  0.487321  0.522999  0.695293  0.867587   

                          
           75%       max  
key1                      
a     0.237196  0.330469  
b     1.039881  1.212174

## MARK: 컬럼에 여러가지 함수 적용하기
* read_csv 함수로 데이터를 불러온 다음 팁의 비율을 담기 위한 컬럼인 tip_pct를 추가한다

In [64]:
tips = pd.read_csv('examples/tips.csv')

In [65]:
tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [66]:
# total_bill에서 팁의 비율 추가
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [67]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [68]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [69]:
# 컬럼에 따라 다른 함수를 사용해서 집계를 수행하거나 여러 개의 함수를 한 번에 적용하기를 원한다면 쉽고 간단하게 수행 가능
grouped = tips.groupby(['day', 'smoker'])

In [70]:
grouped_pct = grouped['tip_pct']

In [71]:
# 기술 통계에서는 함수 이름을 문자열로 넘기면 됨
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [72]:
# 함수 목록이나 함수 이름을 넘기면 함수 이름을 컬럼으로 하는 DataFrrame을 얻게 된다
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [73]:
# GroupBy 객체에서 자동으로 지정하는 컬럼 이름을 그대로 쓰지 않아도 됨
# lambda 함수는 이름이 '<lambda>'인데 이를 그대로 쓸 경우 알아보기 힘들어진다.
# 이 때, 이름과 함수가 담긴 튜플의 리스트를 넘기면 각 튜플에서 첫 번째 원소가 DataFrame에서 컬럼 이름으로 사용된다
# 2개의 튜플을 가지는 리스트가 순서대로 매핑
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [74]:
# DataFrame은 컬럼마다 다른 함수를 적용하거나 여러 개의 함수를 모든 컬럼에 적용할 수 있다
# tip_pct와 total_bill 컬럼에 대해 동일한 세 가지 통계를 계산한다고 가정하자
functions = ['count', 'mean', 'max']

In [75]:
# 여러 키를 이용한 인덱싱은 더이상 사용되지 않고 list를 대신해서 사용한대요,,, 그렇지만 저는 바보라서 리스트로 어떻게 변환하는 지 모름,,,임니다,,,
result = grouped[['tip_pct', 'total_bill'].agg(functions)

/Users/adorable_hj/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [76]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [77]:
# DataFrame은 계층적인 컬럼을 가지고 있음
# 이는 각 컬럼을 따로 계산한 다음 concat 메서드를 이용하여 keys 인자로 컬럼 이름을 넘겨 이어 붙인 것과 동일
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [78]:
# 이전처럼 컬럼 이름과 메서드가 담긴 튜플의 리스트를 넘기는 것도 가능
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [80]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

/Users/adorable_hj/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [81]:
# 컬럼마다 다른 함수를 적용하고 싶다면 agg 메서드에 컬럼 이름에 대응하는 함수가 들어있는 사전을 넘기면 됨
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [82]:
# 단 하나의 컬럼에라도 여러 개의 함수가 적용되었다면 DataFrame은 계층적인 컬럼을 가지게 됨
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'], 'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

## MARK: 색인되지 않은 형태로 집계된 데이터 반환
* 지금까지 살펴본 모든 예제에서 집계된 데이터 = 유일한 그룹키 조합으로 색인(혹은 계층적 색인)되어 반환
* 하지만 이런 동작을 기대하지 않을 경우, groupby 메서드에 as_index=False를 넘겨 색인되지 않도록 지정할 수도 있음

In [83]:
# 동일한 결과를 얻기 위해 reset_index 메서드를 호출해서 같은 결과를 얻을 수 있다
# 하지만 as_index=False 옵션을 사용하면 불필요한 계산을 피할 수 있다
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [84]:
# 해당 코드를 작성하지 않은 것과 비교. 위의 경우 인덱스가 없어진 것을 볼 수 있음.
tips.groupby(['day', 'smoker']).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

# 3. Apply: 일반적인 분리-적용-병합
* 가장 일반적인 GroupBy 메서드의 목적은 Apply
* apply 메서드는 객체를 여러 조각으로 나누고, 전달된 함수를 각 조각에 일괄 적용한 후 이를 다시 합친다.
* 앞서 살펴본 tip 데이터에서 그룹별 상위 5개의 tip_pct 값을 골라보자

In [85]:
# 특정 컬럼에서 가장 큰 값을 가지는 로우를 선택하는 함수 작성
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [168]:
top(tips)

,total_bill,tip,smoker,day,time,size,tip_pct
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [170]:
# 흡연자(smoker) 그룹에 대해 함수(top)를 apply
# 나뉘어진 DataFrame의 각 부분에 모두 적용
# pandas.concat을 이용하여 하나로 합쳐진 다음 그룹 이름표가 붙음
# 결과는 계층적 색인을 가지게 되고 내부 색인은 원본 DataFrame의 색인값을 가지게 됨
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [171]:
tips.groupby('smoker').agg(top)

ValueError: Shape of passed values is (7, 2), indices imply (6, 2)

In [182]:
# 만약 apply 메서드로 넘실 함수가 추가적인 인자를 받는다면 함수 이름 뒤에 붙여서 넘겨주면 됨
tips.groupby(['smoker', 'day']).apply(top, n=3, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  223       15.98   3.00     No   Fri   Lunch     3  0.187735
            91        22.49   3.50     No   Fri  Dinner     2  0.155625
            94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  23        39.42   7.58     No   Sat  Dinner     4  0.192288
            59        48.27   6.73     No   Sat  Dinner     4  0.139424
            212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  11        35.26   5.00     No   Sun  Dinner     4  0.141804
            112       38.07   4.00     No   Sun  Dinner     3  0.105070
            156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 141       34.30   6.70     No  Thur   Lunch     6  0.195335
            85        34.83   5.17     No  Thur   Lunch     4  0.148435
            142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  96        27.28   4.00    Yes   Fri  Dinner     2  0.146628
            90        28.97   3.00    Yes   Fri  Dinner     2  0.103555
            95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  207       38.73   3.00    Yes   Sat  Dinner     4  0.077459
            102       44.30   2.50    Yes   Sat  Dinner     3  0.056433
            170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  180       34.65   3.68    Yes   Sun  Dinner     4  0.106205
            184       40.55   3.00    Yes   Sun  Dinner     2  0.073983
            182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 192       28.44   2.56    Yes  Thur   Lunch     2  0.090014
            83        32.68   5.00    Yes  Thur   Lunch     2  0.152999
            197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

### GroupBy 객체에 describe 메서드 호출
* describe 같은 메서드를 호출하면 GroupBy 내부적으로 다음과 같은 단계를 수행
<pre>
    f = lambda x: x.describe()
    grouped.apply(f)
</pre>

## MARK: 그룹 색인 생략하기
* 이전 예제들에서 반환된 객체는 원본 객체의 각 조각에 대한 색인과 그룹 키가 계층적 색인으로 사용됨을 볼 수 있음
* 이런 결과는 groupby 메서드에 group_key=False를 넘겨 막을 수 있음

In [89]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [90]:
# group_keys=False 옵션을 사용하지 않은 결과
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

## 변위치 분석과 버킷 분석
* 8장 내용: pandas의 cut과 qcut 메서드를 사용해 선택한 크기만큼 혹은 표본 변위치에 따라 데이터를 구분
* 이 함수들을 groupby와 조합하면 데이터 묶음에 대해 변위치 분석이나 버킷 분석을 매우 쉽게 수행할 수 있음
* 임의의 데이터 묶음을 cut을 이용해서 등간격 구간으로 나누어보자

In [204]:
frame = pd.DataFrame({'data1': np.random.randn(1000), 
                      'data2': np.random.randn(1000)})

In [205]:
frame

,data1,data2
0,0.056174,0.196201
1,0.746100,-1.088264
2,-0.663193,-0.124169
3,0.258071,0.893077
4,-1.736900,-0.405163
...,...,...
995,0.719504,-0.111704
996,1.079707,1.426733
997,-0.172022,0.344423
998,0.962022,0.740545


In [206]:
quartiles = pd.cut(frame.data1, 4)

In [207]:
quartiles

0       (-1.405, 0.104]
1        (0.104, 1.614]
2       (-1.405, 0.104]
3        (0.104, 1.614]
4      (-2.921, -1.405]
             ...       
995      (0.104, 1.614]
996      (0.104, 1.614]
997     (-1.405, 0.104]
998      (0.104, 1.614]
999     (-1.405, 0.104]
Name: data1, Length: 1000, dtype: category
Categories (4, interval[float64]): [(-2.921, -1.405] < (-1.405, 0.104] < (0.104, 1.614] < (1.614, 3.124]]

In [208]:
quartiles[:10] # 엥 왜 나는 다 똑같은 값?

0     (-1.405, 0.104]
1      (0.104, 1.614]
2     (-1.405, 0.104]
3      (0.104, 1.614]
4    (-2.921, -1.405]
5    (-2.921, -1.405]
6     (-1.405, 0.104]
7     (-1.405, 0.104]
8      (0.104, 1.614]
9      (1.614, 3.124]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.921, -1.405] < (-1.405, 0.104] < (0.104, 1.614] < (1.614, 3.124]]

In [99]:
# cut에서 반환된 Categorical 객체는 바로 groupby로 넘길 수 있다
# data2 컬럼에 대한 몇 가지 통계를 다음과 같이 계산할 수 있다
def get_stats(group):
    return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}

In [210]:
frame

,data1,data2
0,0.056174,0.196201
1,0.746100,-1.088264
2,-0.663193,-0.124169
3,0.258071,0.893077
4,-1.736900,-0.405163
...,...,...
995,0.719504,-0.111704
996,1.079707,1.426733
997,-0.172022,0.344423
998,0.962022,0.740545


In [212]:
grouped = frame.data2.groupby(quartiles)

In [213]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-2.921, -1.405]",-2.082670,3.102742,80.0,0.042250
"(-1.405, 0.104]",-2.879737,3.352430,464.0,0.047630
"(0.104, 1.614]",-3.073129,4.262020,410.0,0.143881
"(1.614, 3.124]",-1.944913,3.099841,46.0,-0.129319


In [102]:
# 위는 등간격 버킷, 표번 변위치에 기반하여 크기가 같은 버킷을 계산하려면 qcut을 사용
# 혹은 labels=False를 넘겨 변위치 숫자를 구할 수도 있음
grouping = pd.qcut(frame.data1, 10, labels=False)

In [103]:
grouped = frame.data2.groupby(grouping)

In [104]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.603705,3.413642,100.0,-0.027449
1,-2.660120,2.443745,100.0,0.161213
2,-2.669564,2.529981,100.0,-0.047568
3,-2.869714,2.507990,100.0,0.003079
4,-2.227987,2.280495,100.0,0.095797
5,-2.200472,3.693055,100.0,-0.178203
6,-2.321055,2.459952,100.0,0.077004
7,-2.097995,2.083279,100.0,-0.148680
8,-3.294503,3.181250,100.0,-0.004754


## MARK: 그룹에 따른 값으로 결측치 채우기
* 누락된 데이터를 정리할 때면 어떤 경우에는 dropna를 사용하여 데이터를 살펴보고 걸러내기도 함
* 어떤 경우에는 누적된 값을 고정된 값이나 혹은 데이터로부터 도출된 어떤 값으로 채우고 싶을 때도 있음
* 이런 경우 fillna 메서드를 사용
* 누락된 값을 평균값으로 대체하는 예제를 살펴보자

In [108]:
s = pd.Series(np.random.randn(6))

In [109]:
s

0    1.852615
1    0.084501
2   -0.367833
3    2.033792
4    0.318513
5    0.171234
dtype: float64

In [110]:
s[::2] = np.nan

In [111]:
# 0, 2, 4 이런 식으로 스텝 2만큼 Nan 값 넣어주기
s

0         NaN
1    0.084501
2         NaN
3    2.033792
4         NaN
5    0.171234
dtype: float64

In [112]:
s.fillna(s.mean())

0    0.763176
1    0.084501
2    0.763176
3    2.033792
4    0.763176
5    0.171234
dtype: float64

In [113]:
# 그룹별로 채워넣고 싶은 값이 다르다?
# 추측했듯이 데이터를 그룹으로 나누고 apply 함수를 사용해서 각 그룹에 대해 fillna를 적용하면 됨
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']

In [114]:
# ['East'] * 4는 ['East'] 리스트 안에 네 벌의 원소를 이어 붙임
# 리스트를 더하면 각 리스트를 이어붙일 수 있음
group_key = ['East'] * 4 + ['West'] * 4

In [115]:
data = pd.Series(np.random.randn(8), index=states)

In [116]:
data

Ohio          0.634188
New York      0.547799
Vermont       1.746138
Florida      -1.997949
Oregon       -0.382384
Nevada       -0.245356
California   -1.278333
Idaho        -1.601394
dtype: float64

In [118]:
# 몇몇 값을 결측치로 만들어보자
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [119]:
data

Ohio          0.634188
New York      0.547799
Vermont            NaN
Florida      -1.997949
Oregon       -0.382384
Nevada             NaN
California   -1.278333
Idaho              NaN
dtype: float64

In [120]:
data.groupby(group_key).mean()

East   -0.271988
West   -0.830359
dtype: float64

In [121]:
fill_mean = lambda g: g.fillna(g.mean())

In [122]:
data.groupby(group_key).apply(fill_mean)

Ohio          0.634188
New York      0.547799
Vermont      -0.271988
Florida      -1.997949
Oregon       -0.382384
Nevada       -0.830359
California   -1.278333
Idaho        -0.830359
dtype: float64

In [123]:
# 그룹에 따라 미리 정의된 다른 값으로 채워 넣어야 할 경우
# 각 그룹은 내부적으로 name이라는 속성을 가지고 있으므로 이를 이용하면 됨
fill_values = {'East': 0.5, 'West': -1}

In [124]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [125]:
data.groupby(group_key).apply(fill_func)

Ohio          0.634188
New York      0.547799
Vermont       0.500000
Florida      -1.997949
Oregon       -0.382384
Nevada       -1.000000
California   -1.278333
Idaho        -1.000000
dtype: float64

## MARK: 예제- 랜덤 표본과 순열
* 대용량의 데이터를 몬테카를로 시뮬레이션이나 다른 애플리케이션에서 사용하기 위해 랜덤 표본을 뽑아낸다고 해 보자.
* 뽑아내는 방법은 여러 가지 존재, 여기서는 Series의 sample 메서드를 사용

In [126]:
# 트럼프 카드 덱 만들기
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in suits: # suits를 만들어두고 왜 그냥 리스트 써요 글쓴이 아저씨?
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index=cards)

In [127]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [214]:
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64

In [128]:
# 5장의 카드를 뽑기 위해 다음 코드를 작성
def draw(deck, n=5):
    return deck.sample(n)

In [129]:
draw(deck)

3D     3
KH    10
6D     6
9S     9
QD    10
dtype: int64

In [130]:
# 각 세트별로 2장의 카드를 무작위로 뽑고 싶다고 가정하자
# 세트는 각 카드 이름의 마지막 글자이므로 이를 이용해서 그룹을 나누고 apply를 사용하자
get_suit = lambda card: card[-1] # 마지막 글자 추출

In [131]:
deck.groupby(get_suit).apply(draw, n=2)

C  8C      8
   KC     10
D  10D    10
   AD      1
H  4H      4
   8H      8
S  3S      3
   9S      9
dtype: int64

In [132]:
# 아래와 같은 방법으로 각 세트별 2장의 카드를 무작위로 뽑을 수도 있음
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

3C      3
6C      6
7D      7
10D    10
6H      6
5H      5
2S      2
JS     10
dtype: int64

## MARK: 예제- 그룹 가중 평균과 상관관계
* groupby를 나누고 적용하고 합치는 패러다임에서 DataFrame의 컬럼 간 연산이나 두 Series 간의 연산은 일상적인 일
* 그룹 키와 값 그리고 어떤 가중치를 갖는 데이터 묶음을 살펴보자

In [136]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})

In [137]:
df

,category,data,weights
0,a,-0.266836,0.709724
1,a,-0.661828,0.984711
2,a,-1.523492,0.179862
3,a,0.144908,0.596620
4,b,1.014737,0.474530
5,b,0.680897,0.480411
6,b,0.871422,0.808255
7,b,0.704799,0.317945


In [138]:
grouped = df.groupby('category')

In [139]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [140]:
grouped.apply(get_wavg)

category
a   -0.416304
b    0.834663
dtype: float64

In [141]:
# 야후! 파이낸스에서 가져온 몇몇 주식과 S&P 500지수(종목 코드 SPX)의 종가 데이터를 살펴보자
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True, index_col=0)

In [142]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [143]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [144]:
# 퍼센트 변화율로 일일 수익률을 계산하여 연간 SPX 지수와의 상관관계를 살펴보는 일은 흥미로울 수 있는데 아래와 같이 구할 수 있다
# 우선 'SPX' 컬럼과 다른 컬럼의 상관관계를 계산하는 함수를 만든다
spx_corr = lambda x: x.corrwith(x['SPX'])

In [145]:
# 그리고 pct_change 함수를 이용해서 close_px의 퍼센트 변화율을 계산
rets = close_px.pct_change().dropna()

In [146]:
get_year = lambda x: x.year

In [147]:
by_year = rets.groupby(get_year)

In [148]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [149]:
# 두 컬럼 간의 상관관계를 계산하는 것도 가능
# 애플과 마이크로소프트 주가의 연관 상관관계
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

# 4. 피벗 테이블과 교차 일람표
<pre> <b>피벗테이블</b>
 * 스프레드시트 프로그램과 그 외 다른 데이터 분석 소프트웨어에서 흔히 볼 수 있는 데이터 요약 도구
 * 피벗테이블은 데이터를 하나 이상의 키로 수집해서 어떤 키는 로우에, 어떤 키는 컬럼에 나열해서 데이터를 정렬
</pre>
* pandas에서 피벗테이블은 이 장에서 설명했던 groupby 기능을 사용해서 계층적 색인을 활용한 재형성 연산 사용을 가능하게 함
* pandas 모듈의 최상위 함수로, pivot_table() 메서드 존재
* groupby를 위한 편리한 인터페이스를 제공하기 위해 pivot_table은 마진이라고 하는 부분합을 추가할 수 있는 기능 제공

In [150]:
# 요일(day)과 흡연자(smoker) 집단에서 평균을 구해보자
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [152]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [153]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [215]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [151]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [155]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'], columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706